In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

MessageError: ignored

In [ ]:
import pandas as pd
df = pd.read_csv('/content/gdrive/My Drive/Mestrado/second_level_preprocessed.csv')
#df = pd.read_csv('/content/gdrive/My Drive/Mestrado/pre_processing_important.csv')

In [ ]:
df.head()

In [ ]:
# extra preprocessing

df['text_processed'] = df['text_processed'].str.lower()

In [ ]:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer


#using the count vectorizer
count = CountVectorizer()
word_count=count.fit_transform(df['text_processed'])

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count)
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=count.get_feature_names(),columns=["idf_weights"])


#inverse document frequency
df_idf.sort_values(by=['idf_weights'])

#tfidf
tf_idf_vector=tfidf_transformer.transform(word_count)
feature_names = count.get_feature_names()

first_document_vector=tf_idf_vector[1]
df_tfifd= pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])

df_tfifd.sort_values(by=["tfidf"],ascending=True)

In [ ]:
do_not_remove = {"no", "nor", "not", "few", "very",  "don'", "don't",  "'aren'",
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't"}

In [ ]:
temp = df_tfifd.to_dict()

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stopwords_nltlk = list(set(stopwords.words('english')))

ultimate = []
for word in stopwords_nltlk:
  if word in temp['tfidf'].keys():
    if temp['tfidf'][word] == 0 and word not in do_not_remove:
      ultimate.append(word)

#### add ultimate as stopwords

In [ ]:
df.columns

In [ ]:
df['text'][111]

In [ ]:
import nltk
nltk.download('omw-1.4')

In [ ]:
import string
regular_punct = list(string.punctuation)
def remove_punctuation(text,punct_list):
    for punc in punct_list:
        if punc in text:
            text = text.replace(punc, ' ' )
    return text.strip()

df['punct'] = df['text_processed'].apply(lambda x : remove_punctuation(x, regular_punct)).apply(lambda x : " ".join(x.split()))



from nltk.stem.porter import PorterStemmer

porter_stemmer = PorterStemmer()

def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

df['porter_original'] = df['text_processed'].apply(stem_sentences)
df['porter_punct'] = df['punct'].apply(stem_sentences)


import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

#example text text = 'What can I say about this place. The staff of these restaurants is nice and the eggplant is not bad'

class Splitter(object):
    """
    split the document into sentences and tokenize each sentence
    """
    def __init__(self):
        self.splitter = nltk.data.load('tokenizers/punkt/english.pickle')
        self.tokenizer = nltk.tokenize.TreebankWordTokenizer()

    def split(self,text):
        """
        out : ['What', 'can', 'I', 'say', 'about', 'this', 'place', '.']
        """
        # split into single sentence
        sentences = self.splitter.tokenize(text)
        # tokenization in each sentences
        tokens = [self.tokenizer.tokenize(sent) for sent in sentences]
        return tokens


class LemmatizationWithPOSTagger(object):
    def __init__(self):
        pass
    def get_wordnet_pos(self,treebank_tag):
        """
        return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v) 
        """
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            # As default pos in lemmatization is Noun
            return wordnet.NOUN

    def pos_tag(self,tokens):
        # find the pos tagginf for each tokens [('What', 'WP'), ('can', 'MD'), ('I', 'PRP') ....
        pos_tokens = [nltk.pos_tag(token) for token in tokens]

        # lemmatization using pos tagg   
        # convert into feature set of [('What', 'What', ['WP']), ('can', 'can', ['MD']), ... ie [original WORD, Lemmatized word, POS tag]
        pos_tokens = [ [(word, lemmatizer.lemmatize(word,self.get_wordnet_pos(pos_tag)), [pos_tag]) for (word,pos_tag) in pos] for pos in pos_tokens]
        return pos_tokens

lemmatizer = WordNetLemmatizer()
splitter = Splitter()
lemmatization_using_pos_tagger = LemmatizationWithPOSTagger()

#step 1 split document into sentence followed by tokenization

def lemmatise(text):
  tokens = splitter.split(text)
  #step 2 lemmatization using pos tagger 
  lemma_pos_token = lemmatization_using_pos_tagger.pos_tag(tokens)
  ans = ""
  for sentence in lemma_pos_token:
    for word in sentence:
      ans += word[1] + " "

  return ans[:-1]


df['lemma_original'] = df['text_processed'].apply(lemmatise)
df['lemma_punct'] = df['text_processed'].apply(lemmatise)

In [ ]:
df

In [ ]:
mappings = {
    '0': 'information_support',
    '1': 'information_support',
    '2': 'information_support',

    '3':  'emotional_support',
    '4': 'emotional_support',
    '5': 'emotional_support',
    '6': 'emotional_support',
    '7': 'emotional_support',

    '8' : 'esteem_support',
    '9': 'esteem_support',
    
    '10': 'network_support',
    '11': 'network_support',

    #'12': 'perform_direct_task',
    #'13': 'express_willingness',

    '14': 'seeking_support',
    '15': 'seeking_support',

    '16': 'gratitude',
    '17': 'congratulations',
    '18' : 'sharing_personal_experiences' 

}


mappings_first = {
    '0': 'offering_support',
    '1': 'offering_support',
    '2': 'offering_support',

    '3':  'offering_support',
    '4': 'offering_support',
    '5': 'offering_support',
    '6': 'offering_support',
    '7': 'offering_support',

    '8' : 'offering_support',
    '9': 'offering_support',
    
    '10': 'offering_support',
    '11': 'offering_support',

    '12': 'offering_support',
    '13': 'offering_support',

    '14': 'seeking_support',
    '15': 'seeking_support',

    '16': 'group_interactions',
    '17': 'group_interactions',
    '18' : 'group_interactions' 
}


def do(labels):
  labels = labels.split(',')
  new_labels = []
  for label in labels:
    if label in mappings_first:
      new_labels.append(mappings_first[label])
  return list(set(new_labels))


In [ ]:
#df.labels = df.labels.apply(do)

In [ ]:
df = df[df['labels'].apply(len) > 0]
### this is required due to the stratified algorithm split used
df = df.reset_index(drop=True)

In [ ]:
df.shape

In [ ]:
#df['labels'] = df['labels'].apply(do)

In [ ]:
type(df['labels'][0])

In [ ]:
#from ast import literal_eval
#df['labels'] = df['labels'].apply(literal_eval)

In [ ]:
! pip install scikit-multilearn

In [ ]:
! pip install iterative-stratification

In [ ]:

from sklearn.neighbors import NearestNeighbors
import random


def get_minority_samples(X: pd.DataFrame, y: pd.DataFrame, ql=[0.05, 1.]):
    """
    return
    X_sub: pandas.DataFrame, the feature vector minority dataframe
    y_sub: pandas.DataFrame, the target vector minority dataframe
    """

    def get_tail_label(df: pd.DataFrame, ql=[0.05, 1.]) -> list:
      """
      Find the underrepresented targets.
      Underrepresented targets are those which are observed less than the median occurance.
      Targets beyond a quantile limit are filtered.
      """
      irlbl = df.sum(axis=0)
      irlbl = irlbl[(irlbl > irlbl.quantile(ql[0])) & ((irlbl < irlbl.quantile(ql[1])))]  # Filtering
      irlbl = irlbl.max() / irlbl
      threshold_irlbl = irlbl.median()
      tail_label = irlbl[irlbl > threshold_irlbl].index.tolist()
      return tail_label

    
    tail_labels = get_tail_label(y, ql=ql)
    print(set(tail_labels))
    index = y[y[tail_labels].apply(lambda x: (x == 1).any(), axis=1)].index.tolist()
    
    X_sub = X[X.index.isin(index)].reset_index(drop = True)
    y_sub = y[y.index.isin(index)].reset_index(drop = True)
    return X_sub, y_sub


def nearest_neighbour(X: pd.DataFrame, neigh) -> list:
    """
    Give index of 10 nearest neighbor of all the instance
    
    args
    X: np.array, array whose nearest neighbor has to find
    
    return
    indices: list of list, index of 5 NN of each element in X
    """
    nbs = NearestNeighbors(n_neighbors=neigh, metric='euclidean', algorithm='kd_tree').fit(X)
    euclidean, indices = nbs.kneighbors(X)
    return indices


def MLSMOTE(X, y, n_sample = 100, neigh=5):
    """
    Give the augmented data using MLSMOTE algorithm
    
    args
    X: pandas.DataFrame, input vector DataFrame
    y: pandas.DataFrame, feature vector dataframe
    n_sample: int, number of newly generated sample
    
    return
    new_X: pandas.DataFrame, augmented feature vector data
    target: pandas.DataFrame, augmented target vector data
    """
    indices2 = nearest_neighbour(X, neigh=5)
    n = len(indices2)
    new_X = np.zeros((n_sample, X.shape[1]))
    target = np.zeros((n_sample, y.shape[1]))
    for i in range(n_sample):
        reference = random.randint(0, n-1)
        neighbor = random.choice(indices2[reference, 1:])
        all_point = indices2[reference]
        nn_df = y[y.index.isin(all_point)]
        ser = nn_df.sum(axis = 0, skipna = True)
        target[i] = np.array([1 if val > 0 else 0 for val in ser])
        ratio = random.random()
        gap = X.loc[reference,:] - X.loc[neighbor,:]
        new_X[i] = np.array(X.loc[reference,:] + ratio * gap)
    new_X = pd.DataFrame(new_X, columns=X.columns)
    target = pd.DataFrame(target, columns=y.columns)
    return new_X, target

In [ ]:
from sklearn.utils import class_weight

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, BatchNormalization, Input
import tensorflow as tf


def create_model_multiclass(input_dim, output_dim):

    inp = Input(input_dim)
    x = Dense(16, activation='relu', kernel_initializer= tf.keras.initializers.HeNormal(), bias_initializer='zeros')(inp)
    x = Dropout(0.3)(x, training=True)
    out = Dense(output_dim, activation='softmax',kernel_initializer= tf.keras.initializers.HeNormal(), bias_initializer='zeros')(x)
    model = Model(inputs = inp, outputs = out)
    
    opt = tf.keras.optimizers.Adam(learning_rate=3e-4)

    #optimizer
    '''
    opt = tf.keras.optimizers.Adam(learning_rate=3e-4)
    # create model
    model = Sequential()
    model.add(Dense(16, input_dim=input_dim, activation='relu', kernel_initializer= tf.keras.initializers.HeNormal(), bias_initializer='zeros'))
    model.add(Dropout(0.3))
    #model.add(BatchNormalization())
    model.add(Dense(output_dim, activation='softmax',kernel_initializer= tf.keras.initializers.HeNormal(), bias_initializer='zeros'))
    '''

    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])
    return model


def create_model_single_class(input_dim, output_dim):
    
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=input_dim, activation='relu', kernel_initializer= tf.keras.initializers.HeNormal(), bias_initializer='zeros'))
    model.add(Dropout(0.3))
    model.add(Dense(8, activation='relu', kernel_initializer= tf.keras.initializers.HeNormal(), bias_initializer='zeros'))
    model.add(Dense(output_dim, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def create_model_multi_label():
    #optimizer
    opt = tf.keras.optimizers.Adam(learning_rate=3e-4)
    # create model
    model = Sequential()
    model.add(Dense(16, input_dim=(None, ), activation='relu', kernel_initializer= tf.keras.initializers.HeNormal(), bias_initializer='zeros'))
    model.add(Dropout(0.3))
    model.add(Dense(6, activation='softmax',kernel_initializer= tf.keras.initializers.HeNormal(), bias_initializer='zeros'))
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
!pip install arff

In [ ]:
from sklearn.base import TransformerMixin, BaseEstimator

class Debug(BaseEstimator, TransformerMixin):

    def transform(self, X):
        print("Shape of Pre-processed Data:", X.shape)
        print(type(X))
        return X

    def fit(self, X, y=None, **fit_params):
        return self

from scipy import sparse
class SparserTransformer(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return sparse.csr_matrix(X)

class DenseTransformer(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()


In [ ]:
!pip uninstall scikit-learn -y
!pip install scikit-learn==0.24.1 # in order to use multisklearn classifiers

In [ ]:
#### to use label network embeddings
!pip install networkx
!git clone https://github.com/thunlp/OpenNE/
!pip install -e OpenNE/src

In [ ]:
#!pip install textattack

In [ ]:
#! pip install tensorflow_text

In [ ]:
#from textattack.augmentation import CLAREAugmenter
#clare_aug = CLAREAugmenter(pct_words_to_swap=0.2, transformations_per_example=2)

In [ ]:
%%time
#clare_aug.augment("This is simply a simple text")
#clare_aug.augment("This is simply a simple text")

In [ ]:
text = "This is simply a test. And thi is another phrase"

In [ ]:
%%time
from nltk.tokenize import sent_tokenize, word_tokenize 


def augment_sample(text, augmenter, number):
  n = len(sent_tokenize(text))

  result = [''] * number


  for sentence in sent_tokenize(text):
    sentence_augments = augmenter.augment(sentence)
    if len(sentence_augments) < number:
      diff = number - len(sentence_augments)
      sentence_augments.extend([sentence] * diff)

    for i, (r, s) in enumerate(zip(result, sentence_augments)):
      result[i] = r + s 

  return result


In [ ]:
import nltk
from nltk.tag import pos_tag
from nltk import sent_tokenize
from nltk.corpus import wordnet

def get_synonym_for_word(word):
    """returns the synonym given word if found, otherwise returns the same word"""
    
    synonyms = []
    for syn in wordnet.synsets(word):

        for l in syn.lemmas():
            synonyms.append(l.name())
    synonyms = [syn for syn in synonyms if syn!=word]
    if len(synonyms) == 0:
        return word
    else:
        return synonyms[0]



def augment_sentence_wordnet(sentence, filters=['NN', 'JJ']):
    """Augments words in sentence which are filtered by pos tags"""
    
    pos_sent = pos_tag(sentence.split())
    new_sent = []
    for word,tag in pos_sent:
        if tag in filters:
            new_sent.append(get_synonym_for_word(word))
        else:
            new_sent.append(word)
            
    return " ".join(new_sent)


def augment_data(data, target):
    """Creates augmented data using wordnet synonym imputation."""
    
    aug_data = []
    aug_target = []
    for row, t in zip(data, target):
        aug_row = []
        row_sents = sent_tokenize(row)
        #print("row_sents", row_sents)
        for line in row_sents:
            line = augment_sentence_wordnet(line)
            aug_row.append(line)
        row_sents = " ".join(aug_row)
        
        #print(row_sents)
        aug_data.append(row)
        aug_data.append(row_sents)
        aug_target.append(t)
        aug_target.append(t)
        #print(len(aug_data))
    return aug_data, aug_target

In [ ]:
##################### This part is meant to perform some data augmentation ################


def textattack_data_augment(data, target, texattack_augmenter, number):

  '''
    generic function to perform data augmentation
    This needs to receive the textual data rather than the vectorized one
  '''

  aug_data = []
  aug_label = []

  for text, label in zip(data, target):
    
    #### random number [0,2] and perform data augmentation with 1/3 probability
    if random.randint(0,2) != 1:
      aug_data.append(text)
      aug_label.append(label)
      continue

    ### perform data augmentation of that label instance
    #aug_list = texattack_augmenter.augment(text)
    aug_list = augment_sample(text, texattack_augmenter, number)
    aug_data.append(text)
    aug_label.append(label)
    aug_data.extend(aug_list)
    aug_label.extend([label]*len(aug_list))

  return aug_data, aug_label

In [ ]:
########### paraphrase ##############
!pip install transformers
!pip install sentencepiece

In [ ]:
#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
#model = AutoModelForSeq2SeqLM.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
#tokenizer = AutoTokenizer.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
#import torch
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print ("device ",device)
#model = model.to(device)
# Diverse Beam search


In [ ]:
def paraphrase(context):

  text = "" + context + " </s>"
  encoding = tokenizer.encode_plus(text,max_length =128, padding=True, return_tensors="pt")
  input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
  model.eval()
  diverse_beam_outputs = model.generate(
      input_ids=input_ids,attention_mask=attention_mask,
      max_length=128,
      early_stopping=True,
      num_beams=10,
      num_beam_groups = 5,
      num_return_sequences=2,
      diversity_penalty = 0.70
  )

  ans = []
  for beam_output in diverse_beam_outputs:
      sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      ans.append(sent[len("'paraphrasedoutput:") : ])
  return ans

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize 
import nltk
import numpy as np
nltk.download('punkt')

def paraphrase_sample(text):
  n = len(sent_tokenize(text))

  result = []
  smallest = 1000

  for sentence in sent_tokenize(text):
    sentence_augments = paraphrase(sentence)

   
    smallest = min(smallest, len(sentence_augments))
    result.append(sentence_augments[0: smallest])


  result = np.array(result).T
  ans = []
  for i in range(len(result)):
    ans.append(' '.join(result[i]))
    #result[i] = ' '.join(result[i])
      
  return ans

In [ ]:
##################### This part is meant to perform some data augmentation ################


def paraphrase_augmentation(data, target):

  '''
    generic function to perform data augmentation
    This needs to receive the textual data rather than the vectorized one
  '''

  aug_data = []
  aug_label = []

  for text, label in zip(data, target):
    
    #### random number [0,2] and perform data augmentation with 1/3 probability
    if random.randint(0,2) != 1:
      aug_data.append(text)
      aug_label.append(label)
      continue

    ### perform data augmentation of that label instance
    #aug_list = texattack_augmenter.augment(text)
    aug_list = paraphrase_sample(text)
    aug_data.append(text)
    aug_label.append(label)
    aug_data.extend(aug_list)
    aug_label.extend([label]*len(aug_list))

  return aug_data, aug_label

In [ ]:
!pip install textstat

In [ ]:
df['text'][0]

In [ ]:
from nltk import word_tokenize
import nltk
nltk.download('punkt')

In [ ]:
!pip install gensim

In [ ]:
import gensim
from gensim.models import word2vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import gensim.downloader

In [ ]:
def get_embedding_result(word, embedding):
  return embedding[word] if word in embedding else 1

In [ ]:
def get_gloVe(X_train, X_test, embeddings_index):
  X_train_vector = []
  for text in X_train:
    words =  [get_embedding_result(word, embeddings_index) for word in word_tokenize(text)]
    X_train_vector.append(np.average(words, axis = 0))

  X_test_vector = []
  for text in X_test:
    words =  [get_embedding_result(word, embeddings_index) for word in word_tokenize(text)]
    X_test_vector.append(np.average(words, axis = 0))

    
  return X_train_vector, X_test_vector

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def get_gloVe_tf_idf(X_train, X_test, embeddings_index):
  tfidf = TfidfVectorizer()
  tfidf.fit(X_train)
  idf_dict = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

  train_vector = []
  for text in X_train:
    weights = [idf_dict.get(word, 1) for word in word_tokenize(text)]
    words =  [get_embedding_result(word, embeddings_index) for word in word_tokenize(text)]
    train_vector.append(np.average(words, axis = 0, weights = weights))

  test_vector = []
  for text in X_test:
    weights = [idf_dict.get(word, 1) for word in word_tokenize(text)]
    words =  [get_embedding_result(word, embeddings_index) for word in word_tokenize(text)]
    test_vector.append(np.average(words, axis = 0, weights = weights))

  return train_vector, test_vector


In [ ]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [1]:
def vectorize_sentences(X):
  a = []
    for x in X:
      embeded_tweets = embed(x).numpy()
      a.append(embeded_tweets)
  return a

IndentationError: ignored

In [ ]:
############### feature engineering ##################


# a) text metadata length of messages and count frequency words
### some of them should be applied to the original text

def count_chars(text):
    return len(text)


def count_words(text):
  return len(text.split())

def count_capital_words(text):
    return sum(map(str.isupper,text.split()))
 

def count_capital_chars(text):
    count=0
    for i in text:
        if i.isupper():
            count+=1
    return count

def count_punctuations(text):
  punctuations='!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~'
  counter = 0
  for i in punctuations:
      counter += 1
  return counter

def count_sent(text):
    return len(nltk.sent_tokenize(text))

def count_unique_words(text):
    return len(set(text.split()))


def get_length_features(X):

  features = []
  for instance in X:
    char_count = count_chars(instance)
    word_count = count_words(instance)
    sentence_count = count_sent(instance)
    unique_words = count_unique_words(instance)

    features.append([char_count, word_count, sentence_count, count_capital_chars(instance), count_capital_words(instance), unique_words,\
                     count_punctuations(instance), char_count/word_count, word_count/sentence_count, unique_words/word_count])
    
  return features

#df['avg_wordlength'] = df['char_count']/df['word_count']
#df['avg_sentlength'] = df['word_count']/df['sent_count']
#df['unique_vs_words'] = df['unique_word_count']/df['word_count']


##### sentiment score with vader and textblob #######
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
nltk.download('vader_lexicon')

def calc_sentiment_scores(X):
    sid = SentimentIntensityAnalyzer()
    neg = []
    neu = []
    pos = []
    compound = []

    sentiment_textblob = []
    subjectivity = []

    for text in X:
        sentiments = sid.polarity_scores(text)
        neg.append(sentiments['neg'])
        neu.append(sentiments['neu'])
        pos.append(sentiments['pos'])
        compound.append(sentiments['compound'])
        res = TextBlob(text).sentiment
        sentiment_textblob.append(res[0])
        subjectivity.append(res[1])

    neg = np.array(neg).reshape(-1,1)
    neu = np.array(neu).reshape(-1,1)
    pos = np.array(pos).reshape(-1,1)
    compound = np.array(compound).reshape(-1,1)
    sentiment_textblob = np.array(sentiment_textblob).reshape(-1, 1)
    subjectivity = np.array(subjectivity).reshape(-1, 1)

    return np.concatenate([neg,  pos, compound, sentiment_textblob, subjectivity], axis = 1)


####### NER feature engineering #########

import collections

def utils_lst_count(lst):
    dic_counter = collections.Counter()
    for x in lst:
        dic_counter[x] += 1
    dic_counter = collections.OrderedDict( 
                     sorted(dic_counter.items(), 
                     key=lambda x: x[1], reverse=True))
    lst_count = [ {key:value} for key,value in dic_counter.items() ]
    return lst_count

# https://towardsdatascience.com/text-analysis-feature-engineering-with-nlp-502d6ea9225d
def utils_ner_features(lst_dics_tuples, tag):
    if len(lst_dics_tuples) > 0:
        tag_type = []
        dic_counter = collections.Counter()
        for tuple in lst_dics_tuples:
              dic_counter[tuple[1]]+=1
        return dic_counter[tag]
    else:
        return 0

## extract ner features

def append_ner_features(df, col_tag):
  tags_set = []
  for lst in df[col_tag].tolist():
      for dic in lst:
            tags_set.append(dic[1])
            #for k in dic.keys():
                #tags_set.append(k[1])
  tags_set = list(set(tags_set))
  for feature in tags_set:
      val = df[col_tag].apply(lambda x:   utils_ner_features(x, feature))
      print(feature, val)
      


In [ ]:
import spacy.cli
spacy.cli.download("en_core_web_lg")
## call model
ner = spacy.load("en_core_web_lg")

In [ ]:
def utils_ner_features(lst_dics_tuples, tag):
    if len(lst_dics_tuples) > 0:
        tag_type = []
        dic_counter = collections.Counter()
        for tuple in lst_dics_tuples:
              dic_counter[tuple[1]]+=1
        return dic_counter[tag]
    else:
        return 0


def append_ner_features(df, col_tag, tags_set):
 
  for feature in tags_set:
      df[feature] = df[col_tag].apply(lambda x: 
                                                    utils_ner_features(x, feature))
      
  return df

In [ ]:
df2 = pd.DataFrame()
df2['tags'] = df['text'].apply(lambda x: [ (tag.text, tag.label_) for tag in ner(x).ents] )
tags_set = []
for lst in df2['tags'].tolist():
    for dic in lst:
          tags_set.append(dic[1])

tags_set = list(set(tags_set))

In [ ]:
from collections import defaultdict
nltk.download('averaged_perceptron_tagger')
def PoS_counter(X):

  adjectives = []
  nouns = []
  adverbs = []
  verb = []
  for text in X:
    text = word_tokenize(text)
    counter = defaultdict(int)
    for word in  nltk.pos_tag(text):
      if word[1].startswith('J'):
        counter['adjective'] += 1
      elif word[1].startswith('N'):
        counter['noun'] += 1
      elif word[1].startswith('RB'):
        counter['adverb'] += 1
      elif word[1].startswith('VB'):
        counter['verb'] += 1

    adjectives.append(counter['adjectives'])
    nouns.append(counter['noun'])
    adverbs.append(counter['adverbs'])
    verb.append(counter['verb'])

  adjectives = np.array(adjectives).reshape(-1, 1)
  nouns = np.array(nouns).reshape(-1, 1)
  adverbs = np.array(adverbs).reshape(-1, 1)
  verb = np.array(verb).reshape(-1, 1)


  return np.concatenate([adjectives, nouns, adverbs, verb ], axis = 1)


In [ ]:
!pip install pyLDAvis -qq

In [ ]:
################# topic modeling #################

# Import
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import spacy
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()# Visualise inside a notebook
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

# Read the data
reports = df
reports.head()
reports.info()

# Our spaCy model:
nlp = ner

# Tags I want to remove from the text
removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']
tokens = []

for summary in nlp.pipe(reports['text']):
   proj_tok = [token.lemma_.lower() for token in summary if token.pos_ not in removal and not token.is_stop and token.is_alpha]
   tokens.append(proj_tok)

# Add tokens to new column
reports['tokens'] = tokens
reports['tokens']

# Create dictionary
# I will apply the Dictionary Object from Gensim, which maps each word to their unique ID:
dictionary = Dictionary(reports['tokens'])
print(dictionary.token2id)

# Filter dictionary
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)

# Create corpus
corpus = [dictionary.doc2bow(doc) for doc in reports['tokens']]

# LDA model building
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=50, num_topics=10, workers = 4, passes=10)

# Coherence score using C_umass:
topics = []
score = []
for i in range(1,20,1):
   lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=10, num_topics=i, workers = 4, passes=10, random_state=100)
   cm = CoherenceModel(model=lda_model, corpus=corpus, dictionary=dictionary, coherence='u_mass')
   topics.append(i)
   score.append(cm.get_coherence())
_=plt.plot(topics, score)
_=plt.xlabel('Number of Topics')
_=plt.ylabel('Coherence Score')
plt.show()

# Coherence score using C_v:
topics = []
score = []
for i in range(1,20,1):
   lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=10, num_topics=i, workers = 4, passes=10, random_state=100)
   cm = CoherenceModel(model=lda_model, texts = reports['tokens'], corpus=corpus, dictionary=dictionary, coherence='c_v')
   topics.append(i)
   score.append(cm.get_coherence())
_=plt.plot(topics, score)
_=plt.xlabel('Number of Topics')
_=plt.ylabel('Coherence Score')
plt.show()

# Optimal model
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=100, num_topics=5, workers = 4, passes=100)

# Print topics
lda_model.print_topics(-1)

# Where does a text belong to
lda_model[corpus][0]
reports['text'][0]

# Visualize topics
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_display)

In [ ]:
topic_values = []

for i in range(len(reports['text'])):
  n = [0] * len(lda_model.print_topics(-1))

  for v in lda_model[corpus][i]:
    topic = v[0]
    probability = v[1]

    n[topic] = probability

  topic_values.append(n)

topic_values = np.array(topic_values)

In [ ]:
topic_values

In [ ]:
!pip install textstat

In [ ]:
import multiprocessing
from tqdm import tqdm_notebook
import os
import textstat

In [ ]:


######  calculate the readability score of each row ################
def readability_scores_mp(data):
    result_dict, idx, text = data

  #  flesch_reading_ease =  textstat.flesch_reading_ease(text)
    flesch_kincaid_grade =  textstat.flesch_kincaid_grade(text)
    dale_chall_readability_score =  textstat.dale_chall_readability_score(text) 
    result_dict[idx] = [flesch_kincaid_grade, dale_chall_readability_score]

def calc_readability_scores(df, col_text):
    manager = multiprocessing.Manager()
    result_dict = manager.dict()
    mp_list = [(result_dict, idx, title) for idx, title in enumerate(df[col_text].values)]

    with multiprocessing.Pool(os.cpu_count()) as p:
        r = list(tqdm_notebook(p.imap(readability_scores_mp, mp_list), total=len(mp_list)))

    rows = [result_dict[idx] for idx in range(df[col_text].values.shape[0])]
    return pd.DataFrame(rows).values

In [ ]:
scores = np.array(calc_readability_scores(df, 'text'))

In [ ]:
scores.shape

In [ ]:
from ast import literal_eval
df['labels'] = df['labels'].apply(literal_eval)

In [ ]:
y.shape

In [ ]:
import gensim
import gensim.downloader
#embedding = gensim.downloader.load('glove-wiki-gigaword-300')

In [ ]:
#embeddings = [ gensim.downloader.load('fasttext-wiki-news-subwords-300'), gensim.downloader.load('word2vec-google-news-300'), gensim.downloader.load('glove-wiki-gigaword-100'),  gensim.downloader.load('glove-wiki-gigaword-300'), gensim.downloader.load('glove-twitter-50'), gensim.downloader.load('glove-twitter-200') ]

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize 

def get_embedding_result(word, embedding):
  return embedding[word] if word in embedding else 1


def get_gloVe(X_train, X_test, embeddings_index):
  X_train_vector = []
  for text in X_train:
    words = [embeddings_index[word] for word in word_tokenize(text) if word in embeddings_index and word not in ultimate]
    #words =  [get_embedding_result(word, embeddings_index) for word in word_tokenize(text)]
    X_train_vector.append(np.average(words, axis = 0))

  X_test_vector = []
  for text in X_test:
    words = [embeddings_index[word] for word in word_tokenize(text) if word in embeddings_index and word not in ultimate]
    #words =  [get_embedding_result(word, embeddings_index) for word in word_tokenize(text)]
    X_test_vector.append(np.average(words, axis = 0))

    
  return X_train_vector, X_test_vector


from sklearn.feature_extraction.text import TfidfVectorizer

def get_gloVe_tf_idf(X_train, X_test, embeddings_index):
  tfidf = TfidfVectorizer()
  tfidf.fit(X_train)
  idf_dict = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

  train_vector = []
  for text in X_train:
    weights = [idf_dict.get(word, 1)for word in word_tokenize(text) if word in embeddings_index and word not in ultimate]
    #weights = [idf_dict.get(word, 1) for word in word_tokenize(text)]
    #words =  [get_embedding_result(word, embeddings_index) for word in word_tokenize(text)]
    words = [embeddings_index[word] for word in word_tokenize(text) if word in embeddings_index and word not in ultimate]

    train_vector.append(np.average(words, axis = 0, weights = weights))

  test_vector = []
  for text in X_test:
    weights = [idf_dict.get(word,1 )for word in word_tokenize(text) if word in embeddings_index and word not in ultimate]
    #weights = [idf_dict.get(word, 1) for word in word_tokenize(text)]
    #words =  [get_embedding_result(word, embeddings_index) for word in word_tokenize(text)]
    words = [embeddings_index[word] for word in word_tokenize(text) if word in embeddings_index and word not in ultimate]

    test_vector.append(np.average(words, axis = 0, weights = weights))

  return train_vector, test_vector


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

#### one hote enconding on the labels ########
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['labels'])

#### get the text column

X = df['text']

In [ ]:
X.shape, y.shape

In [ ]:
df.columns

In [ ]:
X.shape, y.shape

In [ ]:
embedding = gensim.downloader.load('fasttext-wiki-news-subwords-300')

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize 
import numpy as np

def average_sentences(X):

    new_X = []
    for instance in X:
      sentences = sent_tokenize(instance)
      current_embedding = embed(sentences).numpy()
      '''
      diff = number -len(current_embedding)
      if diff > 0:
          padding = np.zeros((diff, 512))
          current_embedding = np.concatenate((padding, current_embedding), axis=0)
      '''
      new_X.append(np.mean(current_embedding, axis=0))
      
    
    return np.array(new_X)


from sklearn.feature_extraction.text import TfidfVectorizer

def mean_size(X, X_test):


    tfidf = TfidfVectorizer()
    tfidf.fit(X)
    idf_dict = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

    new_X = []
    for instance in X:
      sentences = sent_tokenize(instance)
      weights = []
      for sentence in sentences:
        weights.append(np.mean([1for word in word_tokenize(sentence)]))

      current_embedding = embed(sentences).numpy()

      new_X.append(np.average(current_embedding, axis=0, weights = weights))

    X_tests = []
    for instance in X_test:
      sentences = sent_tokenize(instance)
      weights = []
      for sentence in sentences:
        weights.append(np.mean([1 for word in word_tokenize(sentence)]))

      current_embedding = embed(sentences).numpy()

      X_tests.append(np.average(current_embedding, axis=0, weights = weights))
      
    
    return np.array(new_X), np.array(X_tests)

In [ ]:
new_y = []
new_X = []
i = 0
for instance in df['labels']:
  current_ine = []
  for label in instance:
    if label in  ['informative-support', 'emotional-support', 'esteem-support', 'network-support']:
      current_ine.append(label)

  if len(current_ine) > 0:
    new_y.append(current_ine)
    new_X.append(df['text'][i])
  i = i + 1


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

#### one hote enconding on the labels ########
mlb = MultiLabelBinarizer()
new_y = mlb.fit_transform(new_y)

In [ ]:
mlb.classes_

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from imblearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier ### does not take into account label correlations
from sklearn.svm import SVC
from sklearn.metrics import classification_report, multilabel_confusion_matrix, accuracy_score, hamming_loss, jaccard_score

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

#from sklearn.multioutput import ClassifierChain

from sklearn.utils import compute_class_weight
from skmultilearn.problem_transform import BinaryRelevance
from keras.wrappers.scikit_learn import KerasClassifier

from skmultilearn.problem_transform import ClassifierChain, LabelPowerset
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from skmultilearn.cluster import NetworkXLabelGraphClusterer
from skmultilearn.cluster import LabelCooccurrenceGraphBuilder
from skmultilearn.ensemble import LabelSpacePartitioningClassifier

msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)


# linear is nice 0.47
# poly 0.37
# rbf 0.34
# sigmoid 0.43

#from skmultilearn.ext import Keras

from skmultilearn.adapt import MLkNN
from sklearn.model_selection import GridSearchCV


################ run the pipeline #####################

#### RF E LR
for train_index, test_index in msss.split(np.array(X), np.array(y)):
  #index = - 1
  #for clf in [BinaryRelevance(), ClassifierChain(), LabelPowerset()]:
   #index = index + 1
   #for tfidf in [True, False]:
    #for embedding_str in  [\
    #   'word2vec-google-news-300'    ,
    #  'fasttext-wiki-news-subwords-300', 
   #   'glove-twitter-50', #done
    #  'glove-twitter-200' # done
    #  ]:

      #embedding = gensim.downloader.load(embedding_str)
    #for other_thing in [BinaryRelevance(), ClassifierChain(), LabelPowerset()]:
     #for tf_idf in [True, False]:

      import warnings
      warnings.simplefilter(action='ignore', category=FutureWarning)
      ####### performing the different splits here #########
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]


      #if tfidf:
      #else:
      #  X_train, X_test = get_gloVe(X_train, X_test, embedding)


      from skmultilearn.problem_transform import BinaryRelevance



      ### model xgboost ###~
      from xgboost import XGBClassifier
      from sklearn.multiclass import OneVsRestClassifier


      
      KERAS_PARAMS = dict(epochs=15, batch_size=2, verbose=1)
        


      #### model b)
      from skmultilearn.adapt import BRkNNaClassifier # 1 parameter estimation

      ### model c)
     
      #clf = GridSearchCV(MLkNN(), parameters, scoring=score)

      ### model d) # alternative to for example classifier chain
      from sklearn.naive_bayes import GaussianNB
     

      #classifier = RakelD(
      #    base_classifier= SVC(class_weight='balanced', random_state=42, kernel=kernels[0], C=2),
      #    base_classifier_require_dense=[True, True],
      #    labelset_size=4
      #)

      ### model e) # another alternative
      from skmultilearn.ensemble import MajorityVotingClassifier
      from skmultilearn.cluster import FixedLabelSpaceClusterer
      from skmultilearn.problem_transform import ClassifierChain
      from sklearn.naive_bayes import GaussianNB



      ### labels embeddings #### => nao tou a conseguir por a dar
      import joblib
      import sys
      sys.modules['sklearn.externals.joblib'] = joblib
      #from skmultilearn.embedding import CLEMS, EmbeddingClassifier



      # Compile model
      ###### vectorize the text for training and test seperately #######
      from sklearn.preprocessing import FunctionTransformer


      ### perform data augmentation of the training data prior to the preprocessing phase ###
      #from textattack.augmentation import CheckListAugmenter
      #from textattack.augmentation import EasyDataAugmenter, EmbeddingAugmenter


      #X_min, y_min = get_minority_samples(X_train, pd.DataFrame(y_train, columns=mlb.classes_))
      #print("Size of the minority samples", X_min.shape[0])
      number=1
      #checklist_augmenter = EmbeddingAugmenter(pct_words_to_swap=0.1, transformations_per_example=number)

      #X_train, y_train = paraphrase_augmentation(X_train, y_train)
      
      #X_train, y_train = textattack_data_augment(X_train, y_train, checklist_augmenter, number)

      #X_train, y_train = augment_data(X_train, y_train)

      X_train = np.array(X_train)
      y_train = np.array(y_train)



      from sklearn.neighbors import KNeighborsClassifier

      #print("New dataset ", X_train.shape, "and ", y_train.shape)
      from sklearn.tree import DecisionTreeClassifier
      #classifier = DecisionTreeClassifier()
      from sklearn.naive_bayes import GaussianNB

      from sklearn.neighbors import KNeighborsClassifier
      from sklearn.linear_model import LogisticRegression

      
      from sklearn.naive_bayes import MultinomialNB
      '''
      '''
      parameters = {
        "vect__analyzer": ['word'],
        "vect__ngram_range": [ (1,3)],
        'tfidf__use_idf': [ True,],
        'tfidf__norm': ['l2'],
        "clf__classifier": [SVC()],
        "clf__classifier__kernel": ['linear'],
        "clf__classifier__C": [1],
        "clf__classifier__class_weight": ["balanced"]


          #"classifier":[LogisticRegression(class_weight="balanced")],
        #"classifier__penalty":['l1', 'l2'],
        #"classifier__C": [0.1, 1, 10, 100],
        #"classifier__solver": ['liblinear']
        #"clf__classifier__loss": ["hinge", "squared_hinge"],
        #"clf__classifier__penalty": ['l1', 'l2']
        #"clf__classifier__var_smoothing": [1e-8, 1e-9, 1e-10]
        #"clf__n_neighbors":[3, 5, 10],
        #"clf__weights": ['uniform', 'distance'],
        #"clf__p": [1,2 ]
      }

      parameters2 = {
        "classifier": [ClassifierChain()],
        "classifier__alpha": [1e-2, 1e-3]
      }
      #X_train, X_test = get_gloVe(X_train, X_test, embedding)

      
      parameters3= {
        # DT
        #"criterion": ['gini', 'entropy'],
        #"class_weight": ["balanced"],
        #"ccp_alpha": [0, 0.005, 0.01, 0.02]

        # 

        #"classifier": [GaussianNB()],
        #"classifier__var_smoothing": [1e-2, 1e-3],

        #"n_neighbors":[3, 5, 10],
        #"weights": ['uniform', 'distance'],
        #"p": [1,2] ,


        #"classifier": [SVC()],
        #"classifier__kernel": ['linear'],
        #"classifier__C": [20],
        #"classifier__class_weight": ["balanced"]

        #"classifier":[LogisticRegression(class_weight="balanced")],
        #"classifier__penalty":['l1', 'l2'],
        #"classifier__C": [0.1, 1, 10, 100],
        #"classifier__solver": ['liblinear']


        #"bootstrap": [True, False],
        #"max_depth": [10, 20],
        #"n_estimators": [400, 600],
        #"class_weight":['balanced']


      
        #"classifier__alpha": [1e-2, 1e-3]
        #"vect__ngram_range": ((1, 1), (1, 2)),


        #'tfidf__use_idf': (True, False),
        #'tfidf__norm': ['l2', 'l1'],
        #"clf__n_neighbors":[3, 5, 10],
        #"clf__weights": ['uniform', 'distance'],
        #"clf__p": [1,2 ]
        #'clf__bootstrap': [True, False],
        #'clf__max_depth': [10, 30], #40, 50, 60, 100, None
        #'clf__n_estimators': [ 200, 300] # , 400, 600, 800, 1000
        }

      from sklearn.metrics import make_scorer

      #if tf_idf:
      #  X_train, X_test = get_gloVe_tf_idf(X_train, X_test, embedding)
      #else:
      #  X_train, X_test = get_gloVe(X_train, X_test, embedding)
      #X_train = vectorize_sentences(X_train)
      #X_test = vectorize_sentences(X_test)
      #X_train = np.array(X_train)
      #X_test = np.array(X_test)


            ###### perform feature engineering and append those features #####
      from sklearn.preprocessing import Normalizer, StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler 
      
      X_train_length = get_length_features(X_train)
      X_test_length = get_length_features(X_test)

      X_train_sentiment = calc_sentiment_scores(X_train)
      X_test_sentiment = calc_sentiment_scores(X_test)

      X_train_read = scores[train_index]
      X_test_read = scores[test_index]

      ##### topic values ########

      X_train_topic = topic_values[train_index]
      X_test_topic = topic_values[test_index]

      ######## ner part if we want #####

      X_train_ner = append_ner_features(df2.iloc[train_index], 'tags', tags_set)[tags_set]
      X_test_ner = append_ner_features(df2.iloc[test_index], 'tags', tags_set)[tags_set]

      ########### pos ##############
      X_train_pos = PoS_counter(X_train)
      X_test_pos = PoS_counter(X_test)


      feature_train = np.hstack(( 
          X_train_pos,
          X_train_ner,
          X_train_sentiment,
          X_train_length,
                      X_train_read,
     
                      X_train_topic,
                   
                     ))
      
      normalizer = MinMaxScaler()
      feature_train = normalizer.fit_transform(feature_train)
      feature_train = sparse.csr_matrix(feature_train)


      feature_test = np.hstack(( 
          X_test_pos,
          X_test_ner,
          X_test_sentiment,
          X_test_length,
                      X_test_read,
        
                      X_test_topic ,
                   
                   ))
      
      normalizer = MinMaxScaler()
      feature_test = normalizer.fit_transform(feature_test)
      feature_test = sparse.csr_matrix(feature_test)

      
   


      from sklearn.feature_extraction.text import TfidfVectorizer
      
      #vectorizer = TfidfVectorizer(analyzer="word", ngram_range=(1,3), use_idf=True, norm='l2')
      #X_train = vectorizer.fit_transform(X_train)
      #X_test = vectorizer.transform(X_test)

      X_train  = average_sentences(X_train)
      X_test = average_sentences(X_test)


      
    
      X_train = sparse.hstack((
          sparse.csr_matrix(X_train),
          feature_train
      ))
      X_test = sparse.hstack((
          sparse.csr_matrix(X_test),
          feature_test
      ))

      print(X_train.shape)
      
    
      clf = BinaryRelevance(LogisticRegression(C= 10, penalty='l2', solver='liblinear', class_weight='balanced'))

      
     # text_clf = Pipeline([('vect', CountVectorizer(stop_words=ultimate)),
     #       ('tfidf', TfidfTransformer()),
            #('smote', MLSmote()),
            #('fs', SelectKBest(chi2, k=500)),
            #('dense', DenseTransformer()),
      #      ('clf', BinaryRelevance()),
      #])


      #grid_search = GridSearchCV(clf, parameters, verbose=1, cv=5)
      clf.fit(X_train, y_train)

      #print("val", index)
      print("Best score: %0.3f" % grid_search.best_score_)
      print("Best parameters set:")
      best_parameters = grid_search.best_estimator_.get_params()
      print(parameters.keys())
      for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))


      y_predict = clf.predict(X_test)
      # Find the best parameters for both the feature extraction and the
      # classifier

      ######## APPLY SMOTED IF WANTED  ###################

      #vectorizered_text = text_clf.fit_transform(X_train, y_train)

      ['fasttext-wiki-news-subwords-300',
      'word2vec-google-news-300', #done
      'glove-twitter-50', #done
      'glove-twitter-200' # done
      ]



      
  
      '''
      ### readability scores ###

      X_train_read = scores[X_train.index]
      X_test_read = scores[X_test.index]

      ##### topic values ########

      X_train_topic = topic_values[X_train.index]
      X_test_topic = topic_values[X_test.index]

      ######## ner part if we want #####

      X_train_ner = append_ner_features(df2.iloc[X_train.index], 'tags', tags_set)[tags_set]
      X_test_ner = append_ner_features(df2.iloc[X_test.index], 'tags', tags_set)[tags_set]

      print("ner", X_train_ner.shape, X_test_ner.shape)

      X_train_pos = PoS_counter(X_train)
      X_test_pos = PoS_counter(X_test)
      '''
      #embedding = gensim.downloader.load('glove-twitter-50')

      #vectorizered_text, test_text = get_gloVe_tf_idf(X_train, X_test, embedding)

      #### here for the others
      '''
      X_train = text_clf.fit_transform(X_train)
      test_text = text_clf.transform(X_test)

      print(np.array(X_train).shape, np.array(test_text).shape)
      X_train = pd.DataFrame(data = X_train)
      y_train =  pd.DataFrame(y_train, columns=mlb.classes_)
      '''


      

      #normalizer = MaxAbsScaler ()
      #X_train = normalizer.fit_transform(X_train)
      #X_test = normalizer.fit(X_test)

      #features = sparse.csr_matrix(features)


      '''
      PERCENTAGE = 0 # 0 means that is not applied
      NR_NEIGHBORS = 5

      vectorizered_text = text_clf.fit_transform(X_train, y_train)
      
      df_X = pd.DataFrame(data = vectorizered_text.toarray())
      df_y =  pd.DataFrame(y_train, columns=mlb.classes_)

      X_sub, y_sub = get_minority_samples(df_X, df_y)

      #print("Size of the minority samples", X_sub.shape[0])

      X_res, y_res = MLSMOTE(X_sub, y_sub, int(df_X.shape[0] * PERCENTAGE), NR_NEIGHBORS)
      '''
      ############## train the classifier

      #X_train_augmented = pd.concat([df_X, X_res])
      #classifier.fit(pd.concat([df_X, X_res]), pd.concat([df_y, y_res]))
      #classifier.fit(X_train, y_train)
      ###### perform model validation #####################


      #y_predict = classifier.predict(X_test)

      ### trying monte carlo ####
      '''
      test_text = text_clf.transform(X_test)
      mc_predictions = []

      for i in range(20):
        y_p = classifier.predict(test_text)
        mc_predictions.append(y_p)
      
      accs = []
      for y_p in mc_predictions:
        acc = accuracy_score(y_pred=y_p, y_true=y_test)
        accs.append(acc)

      print("MC accuracy: {:.1%}".format(sum(accs)/len(accs)))
      '''
      #print("embedding", embedding_str)
      #print("tfidf", tfidf)
      from sklearn.metrics import f1_score
      print("accuracy: ", accuracy_score(y_test, y_predict))
      print("hamming loss: ", hamming_loss(y_test, y_predict))
      print( round(hamming_loss(y_test, y_predict),3), round(jaccard_score(y_test, y_predict, average='weighted'),3))
      print( " + ".join(   [str(round(x,2)) for x in f1_score(y_test, y_predict, average=None) ]       ), "(", round(f1_score(y_test, y_predict, average="weighted"),2), ")")
      print("jaccard score: ", jaccard_score(y_test, y_predict, average='weighted'))
      print(multilabel_confusion_matrix(y_test, y_predict))
      print(classification_report(y_test, y_predict))


In [ ]:
mlb.classes_